# Text Mining 
## Hand-in Assignment 1

Student: Elin Benja Dijkstra

Studentnumber: S2696096

Deadline: 20-10-2019

1. The	tutorial	classifies	between only	four	categories	of	the	20newsgroups	data	set.	Change	
your	script	so	that	it	addresses all	20	categories.
DONE

2. Compare	three	classifiers	on	this	multi-class	classification	task,	including	at	least	Naïve	Bayes.
3. Compare	three	type	of	features for	your	classifiers:	counts,	tf,	and	tf-idf.
4. Look	up	the	documentation	of	the	CountVectorizer function	and	experiment	with	different	
values	for	the	following	parameters:	
a. lowercase
b. stop_words
c. analyzer	(in	combination	with	ngram_range)
d. max_features
5. Write	one	script	for	running these	experiments and	printing	the	results.

## Import needed packages

In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

import pandas as pd
import numpy as np
import matplotlib as plt

In [5]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Loading and examining the dataset

To complete the first assignment 1.1) we simply remove the categories = categories so we no longer get a subset but all 20 categories in the dataset.

In [3]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

## Text Tokenization

In [4]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

## Training a classifier

Compare	three	classifiers	on	this	multi-class	classification	task,	including	at	least	Naïve	Bayes.


#### Multinomial Naive Bayes

In [80]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [81]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => rec.autos


In [ ]:
def train_classifier(type_classifier):
    if type_classifier == "NB":
        clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', MultinomialNB()),
        ])
    elif type_classifier == "SVM":
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                  alpha=1e-3, random_state=42,
                                  max_iter=5, tol=None)),
        ])
    elif type_classifier == "Third classifier"
    

## Building a pipeline

In [82]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

The names vect, tfidf and clf (classifier) are arbitrary. We will use them to perform grid search for suitable hyperparameters below. We can now train the model with a single command

In [83]:
text_clf.fit(twenty_train.data, twenty_train.target)  

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

## Evaluation on the test set

In [84]:
twenty_test = fetch_20newsgroups(subset='test',
     categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target) 

0.2782956058588549

In [85]:
# instead of naive bayes use Support vector machine

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(twenty_train.data, twenty_train.target)  
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target) 

0.3382157123834887

In [86]:
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))
print("Confusion matrix: ")
print(metrics.confusion_matrix(twenty_test.target, predicted))

ValueError: Number of classes, 20, does not match size of target_names, 4. Try specifying the labels parameter

## Parameter tuning using grid search

In [ ]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

In [ ]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

In [ ]:
gs_clf.best_score_
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [ ]:
pd.DataFrame(gs_clf.cv_results_)